# Pull NY Times county level data

In [7]:
import pandas as pd
#import sqlalchemy as db

url = 'https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv'
NYTimesDf = pd.read_csv(url, dtype={'fips': object})

# Convert the date field to a date format from string
NYTimesDf['date'] = pd.to_datetime(NYTimesDf['date'])
NYTimesDf['date'] = NYTimesDf['date'].dt.date

NYTimesDf.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061,1,0
1,2020-01-22,Snohomish,Washington,53061,1,0
2,2020-01-23,Snohomish,Washington,53061,1,0
3,2020-01-24,Cook,Illinois,17031,1,0
4,2020-01-24,Snohomish,Washington,53061,1,0


# Get Population Data and aggregate it  to the state level

In [8]:
# Get the population info
CountyPop = pd.read_csv("D:\JP\Covid19\CountyPop.csv", dtype={'fips': object})
CountyPop = CountyPop[["fips", "CountyPopulation2017"]]

# Merge with Covid case and death numbers
NYTimesDf = NYTimesDf.merge(CountyPop, how = 'inner', on = 'fips')

# also create a state aggregate in this form
NYTimesStateRaw = NYTimesDf.groupby(['state', 'date']).sum()
NYTimesStateRaw["CaseDensity"] = (NYTimesStateRaw["cases"] / NYTimesStateRaw["CountyPopulation2017"]) * 100
NYTimesStateRaw["DeathDensity"] = (NYTimesStateRaw["deaths"] / NYTimesStateRaw["CountyPopulation2017"]) * 100
NYTimesStateRaw.tail(3)

cases  deaths  CountyPopulation2017  CaseDensity  \
state   date                                                           
Wyoming 2020-10-30  13028      87                598332     2.177386   
        2020-10-31  13298      87                598332     2.222512   
        2020-11-01  13723      87                598332     2.293543   

                    DeathDensity  
state   date                      
Wyoming 2020-10-30       0.01454  
        2020-10-31       0.01454  
        2020-11-01       0.01454

# Pivot the data for Flourish

In [9]:
#NYTimesDf_sub.pivot_table(index = ['county', 'state','fips'], columns = 'date', values = 'cases')
#NYTimesDf['cases'] = NYTimesDf['cases'].astype(int)
#NYTimesDf.info()
NYTimesStatePivot = NYTimesStateRaw.pivot_table(index = ['state'], columns = 'date', values = 'CaseDensity', fill_value = 0)
#NYTimesPivot.info()
NYTimesStatePivot.head()

date,2020-01-21,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,...,2020-10-23,2020-10-24,2020-10-25,2020-10-26,2020-10-27,2020-10-28,2020-10-29,2020-10-30,2020-10-31,2020-11-01
state,,,,,,,,,,,,,,,,,,,,,
Alabama,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,3.653484,3.701143,3.722932,3.742460,3.764977,3.790604,3.819744,3.846946,3.883074,3.917404
Alaska,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.745420,1.795690,1.866310,1.913210,1.964423,2.011458,2.060919,2.112132,2.171701,2.218332
Arizona,0.0,0.0,0.0,0.0,0.000000,0.000023,0.000023,0.000023,0.000023,0.000023,...,3.355013,3.369064,3.387580,3.401702,3.415426,3.431084,3.448875,3.471473,3.497968,3.519613
Arkansas,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,3.353224,3.394452,3.416677,3.436370,3.471187,3.502880,3.538453,3.575670,3.619067,3.663057
California,0.0,0.0,0.0,0.0,0.000031,0.000015,0.000015,0.000015,0.000015,0.000015,...,2.277949,2.288852,2.295668,2.306663,2.319104,2.329336,2.339971,2.352425,2.361064,2.370241


In [10]:
# export data to excel
import os

print(os.getcwd())

with pd.ExcelWriter('StateLevelCovidData.xlsx') as writer:  
    NYTimesStatePivot.to_excel(writer, sheet_name= 'CaseDensity')

C:\Users\jport
